In [64]:
import json
import os

import dotenv
from aiohttp.typedefs import StrOrURL

from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from tqdm import tqdm

dotenv.load_dotenv()

True

In [65]:
with open('data/Final_TestSet/Final_TestSet.json', 'r', encoding='utf-8') as f:
    dataset_init=json.load(f)
with open('data/Final_Example.json', 'r', encoding='utf-8') as f:
    preliminary_example=json.load(f)

for i in range(0, len(dataset_init)):
    # 检查数据集文件是否一致
    assert dataset_init[i]["ID"] == preliminary_example[i]["ID"] 
    assert dataset_init[i]["question"] == preliminary_example[i]["question"]
    
print("样本数量：",len(dataset_init))
# print("问题类型：",",".join(set([item["problem_type"] for item in dataset_init])))


FROM=0
TO=FROM+512
dataset=dataset_init[FROM:TO]
# INDEX=[22,23,24,28,41,44,59,64,79,80,93,96]
# dataset=[dataset_init[i-1] for i in INDEX]

样本数量： 512


In [66]:
gpt4o=ChatOpenAI(
    api_key=os.getenv("WLAI_API_KEY"),
    base_url=os.getenv("WLAI_BASE_URL"),
    model="gpt-4o",
)

gpt4o.invoke("hello")
set_llm_cache(SQLiteCache(database_path=".langchain.db.bk")) # 

## 预处理


### 修改文件位置
1. 构建prompt
2. 修改题目中文件名位置

In [67]:
from gpt4o import *
for i in range(0, len(dataset)):
    content = d_template[dataset[i]["problem_type"]].format(dataset[i]["question"])
    filenames = extract_filenames(content)
    for filename in filenames:
        content = content.replace(filename, add_path(filename, data_path / 'Final_TestSet/data'))
    dataset[i]["content"]=content

### rag添加文档信息
1.  翻译
2. 从翻译提取函数和库
3. 查询
4. 计划
5. rag

如何衡量：暂时不做




In [68]:
from tool.model import translate_prompt

# 翻译所有问题，已经缓存，所以全量翻译
translation_runnable= translate_prompt | gpt4o | StrOutputParser()
translation_list = translation_runnable.batch([{"text":item["question"]} for item in dataset], config={"max_concurrency":1}, return_exceptions=True)
for i in range(0,len(dataset)):
    dataset[i]["translation"]=translation_list[i]

In [69]:
from tool.model import extract_runnable

# 从翻译中提取出函数和库
extract_list=extract_runnable.batch([{"text":item["translation"]} for item in dataset[:]], config={"max_concurrency":5}, return_exceptions=True)
for i in range(len(extract_list)):
    # print(i+1,extract_list[i])
    dataset[i]["func_extract"]=extract_list[i]

In [70]:
from tool.rag_tool import search_documents_by_help_function

for i,key_work in tqdm(enumerate(extract_list), total=len(extract_list)):
    key_work=key_work if type(key_work) is list else [key_work]
    tmp_set=set()
    for kw in key_work:
        doc=search_documents_by_help_function(kw["function_name"],kw["module_name"])
        tmp_set.add("<api doc>\n" + doc + "\n</api doc>")
    dataset[i]["rag_infos"]=tmp_set

100%|██████████| 512/512 [10:15<00:00,  1.20s/it]


In [74]:
import inspect
from tool.rag_tool import search_documents_in_mutil_keywords

for item in tqdm(dataset[:]):
    question=item["question"]
    tmp_l=[]
    for doc_json,_,_ in search_documents_in_mutil_keywords([], question,5):
        function_name=""
        for key in doc_json:
            if str(key).startswith("Field List > Methods > "):
                function_name=key[22:]
        if function_name!="":
            class_name=doc_json["Section_id"] if "Section_id" in doc_json else doc_json["Section ID"]
        else:
            function_name=doc_json["Section_id"] if "Section_id" in doc_json else doc_json["Section ID"]
            class_name=""
        package_name=doc_json["module"]
        
        help_doc=search_documents_by_help_function(function_name,package_name)
        if len(help_doc)>15000:
            init_len=len(help_doc)
            help_doc=(gpt4o|StrOutputParser()).invoke(f"Below is the documentation generated by the help() function. Extract the main information and reduce the word count to 500 words\n{help_doc}")
            after_len=len(help_doc)
            print(f"{package_name}.{function_name} {init_len} -> {after_len}")
        tmp_l.append(f"function:{function_name}, class:{class_name}, package:{package_name}, doc:'{repr(help_doc)[1:-1] }'")
    item["func_bk"]=tmp_l
    

  3%|▎         | 16/512 [00:25<18:47,  2.27s/it]

cdlib.NodeClustering 35313 -> 4758


  4%|▎         | 18/512 [00:26<12:55,  1.57s/it]

cdlib.AttrNodeClustering 35937 -> 5130


  8%|▊         | 40/512 [01:59<16:13,  2.06s/it]  

...

cdlib.BiNodeClustering 35682 -> 7626


 98%|█████████▊| 504/512 [46:38<02:26, 18.27s/it]

cdlib.NodeClustering 35313 -> 4758


100%|██████████| 512/512 [47:54<00:00,  5.61s/it]

graspologic.DCSBMEstimator 16698 -> 3695


In [75]:
for i in range(len(dataset)):
    dataset[i]["content"]=dataset[i]["content"]+"\n\nThe following function must be used:\n"+"\n".join(dataset[i]["rag_infos"])
    dataset[i]["content"]=dataset[i]["content"]+"\n\nThe following functions can be used optionally:\n"+"\n".join(dataset[i]["func_bk"])

### 添加目标

In [15]:
from tqdm import tqdm
from tool.model import cal_prompt, draw_prompt, tof_prompt

def get_goals(text:str, problem_type:str):
    types=[]
    goals=[]
    if problem_type.startswith("multi"):
        types.extend(problem_type[6:-1].split(", "))
    else:
        types.append(problem_type)

    for t in types:
        if t=="calculations":
            prompt=cal_prompt
        elif t=="True/False":
            prompt=tof_prompt
        elif t=="draw":
            prompt=draw_prompt
        else:
            raise Exception("unknown problem type")
    
        runnable=prompt|gpt4o|StrOutputParser()
        goal=runnable.invoke({"question":text})
        goals.append(goal)
    return goals
        
        
for i in tqdm(range(len(dataset))):
    if dataset[i]["problem_type"].startswith("multi"):
        goals=get_goals(dataset[i]["question"], dataset[i]["problem_type"])
        # print(i+1,goals)
        dataset[i]["goals"]=goals
        dataset[i]["content"]=dataset[i]["content"]+"\n\n"+"\nwe need to answer following question：\n"+"\n".join(goals)
    

100%|██████████| 512/512 [01:41<00:00,  5.03it/s]


## 运行
### 运行agent

In [32]:
from autogen import Cache

def run(item: dict,cache_seed=1):
    content = item["content"]
    item["content"]=content

    # Use DiskCache as cache
    with Cache.disk(cache_path_root="./autogen_cache",cache_seed=cache_seed) as cache:
        chat_result = code_executor_agent.initiate_chat(
            code_writer_agent,
            message=content,
            summary_method='reflection_with_llm',
            summary_args=dict(summary_prompt='only return the code output'),
            cache=cache,
            silent=True,
        )
    # code = extract_python_code(chat_result.chat_history[-3]['content'])[-1]
    code=""
    for i in range(len(chat_result.chat_history)-1, 0, -1):
        l=extract_python_code(chat_result.chat_history[i]['content'])
        if len(l)>0:
            code=l[-1]
            break
    
    answer = chat_result.summary
    if isinstance(answer, dict):
        answer = answer['content']
    item["code"]=code
    item["answer"]=answer
    # item['chat_history']=chat_result.chat_history
    return item


for item in tqdm(dataset[498:]):
    run(item)


  0%|          | 0/14 [00:00<?, ?it/s]


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


  7%|▋         | 1/14 [00:17<03:41, 17.06s/it]


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


 14%|█▍        | 2/14 [00:39<04:06, 20.51s/it]

...

 86%|████████▌ | 12/14 [14:50<07:13, 216.71s/it]


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


 93%|█████████▎| 13/14 [15:02<02:34, 154.78s/it]


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


100%|██████████| 14/14 [15:45<00:00, 67.57s/it] 


### voting

In [ ]:
# for item in [dataset[i-1] for i in [3,6,9,10,12,14,19,21,23,24,28,30,37,40]]:
#     temp_answer=[]
#     for seed in tqdm(range(1,17)):
#         item=run(item,seed)
#         code,answer=item["code"],item["answer"]
#         temp_answer.append(answer)
#     prompt=f"""从下面的不同人表达中，直接返回大部分人想表达的内容，不附带其他信息：\n"""+"\n".join(temp_answer)
#     print(item["ID"],(gpt4o|StrOutputParser()).invoke(prompt))
        
        

## 存储

In [35]:
for i in dataset:
    if type(i['rag_infos']) is set:
        i['rag_infos']=list(i['rag_infos'])

In [34]:
with open('data/SMP_240915_check_2.json', 'w', encoding='utf-8') as f:
    s = json.dumps(dataset, indent=4, ensure_ascii=False)
    f.write(s)

----

In [ ]:
raise Exception("stop")

In [ ]:
with open('data/SMP_240905_check_1.json', 'r', encoding='utf-8') as f:
    tmp_dataset=json.load(f)

In [76]:
id_and_content=[{"ID":i["ID"], "content":i["content"]} for i in dataset]
with open('data/id_and_content_0916_1.json', 'w', encoding='utf-8') as f:
    json.dump(id_and_content, f)

In [57]:
print(dataset[1]["func_bk"][6])

IndexError: list index out of range

In [ ]:


def remove_empty_values(d):
    """
    递归删除字典中的所有空值（包括空字符串、空列表、空字典、None等）
    """
    if not isinstance(d, dict):
        return d
    
    # 使用字典推导式递归遍历字典
    return {k: remove_empty_values(v) for k, v in d.items() if v not in ('', None, [], {}, set(), ())}


for i,key_work in tqdm(enumerate(extract_list), total=len(extract_list)):
    infos=""
    INFO_LIMIT=3000

    for item in key_work:
        if item["function_name"] != "":
            module,function = item['module_name'],item['function_name']
            api_docs=search_documents(function,module,dataset[i]["question"])
            for doc in api_docs[:2]:
                if len(infos)<INFO_LIMIT:
                    if not doc.startswith("no"):
                        doc=json.dumps(remove_empty_values(json.loads(doc)))
                    infos=infos + "\n\n"+doc
        # 没有抽取，尝试用整个问题查询
        else:
            api_docs=search_documents(method_description=dataset[i]["question"])
            for doc in api_docs:
                if len(infos)<INFO_LIMIT:
                    if not doc.startswith("no"):
                        doc=json.dumps(remove_empty_values(json.loads(doc)))
                    infos=infos + "\n\n"+doc
    dataset[i]["rag_infos"]=infos
for i in range(len(dataset)):
    dataset[i]["content"]=dataset[i]["content"]+"\n\n"+dataset[i]["rag_infos"]

In [ ]:
for i in range(len(dataset)):
    rag_infos=dataset[i]["rag_infos"]
    print()
    print(i+1,extract_list[i],[round(t[1]+t[2],2)  for t in rag_infos])
        

In [ ]:
from tool.rag_tool import search_documents_by_help_function

for i in range(0,len(dataset)):
    for item in extract_list[i]:
        fn=item["function_name"].split(".")[-1]
        mo=str(item["module_name"]).lower().strip().split(".")[0]
        print(i, fn, mo,end=" ")
        doc=search_documents_by_help_function(fn, mo)
        if doc:
            print(len(doc))
        else:
            print(None)
            